In [2]:
import pandas as pd
import numpy as np

In [257]:
data1 = pd.read_csv('Twitter_Data/tweets_18_19.csv')
data2 = pd.read_csv('Twitter_Data/tweets_19_20.csv')
data3 = pd.read_csv('Twitter_Data/tweets_20_21.csv')

# data1.info()

In [258]:
from datetime import datetime

def clean_df(df):
    df = df.dropna(subset=['Date'], how='all')
    df = df.fillna('None')
    def is_date(s):
        if len(s) < 11: return False
        return s[0] in '0123456789' and s[4] == '-'
    df['is_date'] = df['Date'].map(lambda i: is_date(i))
    df = df[df['is_date']==True].iloc[:,:len(df.columns)-1]
    date = df['Date'].map(lambda i:datetime.strptime(i[:10].replace('-',''),"%Y%m%d"))
    data = {'date':list(date),'content':list(df.iloc[:,2]),'location':list(df.iloc[:,3]),'num_likes':list(df.iloc[:,4])}

    return pd.DataFrame(data)
    

def is_not_english(s):
    spec_char_count = 0
    for char in s:
        if not (48<=ord(char)<=57 or 65<=ord(char)<=90 or 97<=ord(char)<=122): spec_char_count+=1
        #if ord(u"\u30a0")<=ord(char)<=ord(u"\u30ff") or ord(u"\u2e80")<=ord(char)<=ord(u"\u2eff"): spec_char_count+=1
        if spec_char_count/len(s) > 0.5: return True
    return False

def remove_(s):
    s = s.replace('\n',' ') # remove \n signs
    while 'https://' in s:
        start_idx = s.index('https://')
        if ' ' in s[start_idx:]:
            end_idx = s[start_idx:].index(' ')
            s = s[:start_idx] + s[start_idx+end_idx:]
        else:
            s = s[:start_idx]
    return s
            
def data_clean(df):
    df = clean_df(df)
    df.iloc[:,1] = df.iloc[:,1].map(lambda i: remove_(i))
    df['is_japanese'] = df['content'].map(lambda i: is_not_english(i))
    df = df[df['is_japanese'] == False].reset_index(drop=True)
    
    return df.iloc[:,:4]

In [259]:
data1 = data_clean(data1)
data2 = data_clean(data2)
data3 = data_clean(data3)

In [267]:
data = [data1,data2,data3]

In [268]:
months = ['2021.03','2021.04','2021.05','2021.06','2021.07','2021.08','2021.09','2021.10','2021.11','2021.12','2022.01','2022.02','2022.03','2022.04','2022.05']

for month in months:
    month_data = pd.read_csv('Twitter_Data/{}.csv'.format(month),lineterminator="\n")
    print(month)
    data.append(data_clean(month_data))

data = pd.concat(data,axis=0)
data = data.sort_values(by='date').reset_index(drop=True)

2021.03
2021.04
2021.05
2021.06
2021.07
2021.08
2021.09
2021.10
2021.11


/opt/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2021.12
2022.01


/opt/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2022.02
2022.03
2022.04
2022.05


In [237]:
data.to_csv('twitter_alldata_cleaned.csv',line_terminator='\n')

In [269]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1935564 entries, 0 to 1935563
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   date       datetime64[ns]
 1   content    object        
 2   location   object        
 3   num_likes  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 59.1+ MB


In [270]:
data

,date,content,location,num_likes
0,2018-03-02,Axie Infinity: Terrarium Ethereum-Based Crypto...,Blockchain,0
1,2018-03-05,I just published “You ready to reveal your Axi...,discord.gg/axie,1
2,2018-03-14,"DAppブラウザーアプリ""Cipher Browser""に、ゲームが一挙5タイトル追加。 ...",None,9
3,2018-03-16,@AxieInfinity is proud to be listed in @Cipher...,discord.gg/axie,0
4,2018-03-17,“You ready to reveal your Axies?” by @AxieInfi...,None,0
...,...,...,...,...
1935559,2022-05-04,@peeps026 @Axie44 @AxieInfinity Just awesome!,None,0
1935560,2022-05-04,@AxieLima @AxieInfinity 馃槓,None,0
1935561,2022-05-04,@lexicus143 @AxieInfinity Agree with this. Whe...,None,0
1935562,2022-05-04,@Axie_Chompx4 @AxieInfinity @ChompSquadAxie I'...,"New York, USA",6


In [205]:
import json
keywords = []

for y in ['2018','2019','2020','2021','2022']:
    for m in ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']:
        keyword = {}
        keyword['name'] = '{}.{}.{}'.format(y,m,'Axie')
        keyword['size'] = 100
        keyword['imports'] = []
        keywords.append(keyword)

json.dump(keywords,open('twitter.json','w'),indent=4)

In [230]:
'に' in '[a-zA-Z0-9]'

False

In [247]:
ord(u"\u30a0")<=ord('に')<=ord(u"\u30ff") #or ord(u"\u2e80")<=ord('に')<=ord(u"\u2eff")

False

In [248]:
ord('に')

12395

In [249]:
ord('a')

97

In [250]:
ord('z')

122

In [251]:
ord('0')

48

In [252]:
ord('9')

57

In [253]:
ord('A')

65

In [254]:
ord('Z')

90

In [ ]:
if not (48<=ord(char)<=57 or 65<=ord(char)<=90 or 97<=ord(char)<=122):